# Импорты

In [1]:
import sympy
import plotly.express as px
import pandas

# Входные данные

In [2]:
x = sympy.symbols('x')

f = x ** 2 - sympy.sin(x)
a = 0.5
b = 1.0
x_star = 0.77

# Определение $x_i$ и $f_i$

In [3]:
h = (b - a) / 10
x_points = []
y_points = []
for i in range(11):
    x_points.append(a + i * h)
    y_points.append(float(f.subs(x, x_points[i])))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)
figure.show()

# Вычисление $L_1(x*)$ и $R_1(x*)$ в форме Лагранжа

## Определение $x_0$ и $x_1$ для интерполяции по 2-м точкам

In [4]:
i_2 = 0
for i in range(len(x_points) - 1):
    if x_points[i] <= x_star <= x_points[i + 1]:
        i_2 = i
        break

    if i == len(x_points) - 2:
        raise ValueError("x* вне отрезка [a, b]")
    
print(x_points[i_2], x_star, x_points[i_2 + 1])

figure = px.line(x=x_points, y=y_points, markers=True, color_discrete_sequence=["rgb(120, 120, 120)"])
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=[x_points[i_2]], y=[y_points[i_2]], name="x_0", marker=dict(color="blue", size=5, symbol="circle"))
figure.add_scatter(x=[x_points[i_2 + 1]], y=[y_points[i_2 + 1]], name="x_1", marker=dict(color="blue", size=5, symbol="circle"))

figure.add_scatter(x=[x_star], y=[float(f.subs(x, x_star))], name="x*")

figure.show()

0.75 0.77 0.8


## Вычисление $L_1(x*)$

Интерполяционная формула Лагранжа для 2-х узлов имеет вид:

$\displaystyle L_1(x*) = f(x_i) \frac{x* - x_{i + 1}}{x_i - x_{i + 1}} + f(x_{i + 1}) \frac{x* - x_i}{x_{i + 1} - x_i}$

In [5]:
L_1_Lagrange = f.subs(x, x_points[i_2]) * (x - x_points[i_2 + 1]) / (x_points[i_2] - x_points[i_2 + 1]) + \
    f.subs(x, x_points[i_2 + 1]) * (x - x_points[i_2]) / (x_points[i_2 + 1] - x_points[i_2])
print("L_1 =", L_1_Lagrange.subs(x, x_star))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_1_Lagrange.subs(x, c)) for c in x_points], name="L_1 Lagrange")

figure.show()

L_1 = -0.102425692373810


## Вычисление $R_1(x*)$

### Вычисление 2-й производной от $f(x)$ и определение минимума и максимума на отрезке $[i; i + 1]$

Для определения максимума и минимума $f''(x)$ необходимо решить неравенство $f'''(x) = 0$

In [6]:
f2 = sympy.diff(f, x, 2)
f3 = sympy.diff(f2, x)
extremes_f2 = sympy.solve(sympy.Eq(f3, 0), x)

print("f'' =", f2)
print("f''' =", f3)
print("точки экстремума в f'':", extremes_f2)

f'' = sin(x) + 2
f''' = cos(x)
точки экстремума в f'': [pi/2, 3*pi/2]


Для определения минимума и максимума на отрезке $[i; i + 1]$ необходимо рассмотреть значения функции $f''(x)$ в точках экстремума, которые принадлежат отрезку, и на краях отрезка

In [7]:
x_f2 = [x_points[i_2], x_points[i_2 + 1]] + [float(c) for c in extremes_f2 if c.is_real and x_points[i_2] <= c <= x_points[i_2 + 1]]
y_f2 = [float(f2.subs(x, c)) for c in x_f2]
f2_min = min(y_f2)
f2_max = max(y_f2)

print("минимум f'' =", f2_min)
print("максимум f'' =", f2_max)

figure = px.line()
for point in range(len(x_f2)):
    figure.add_scatter(x=[x_f2[point]], y=[y_f2[point]], marker=dict(color="blue", size=5, symbol="circle"))
figure.update_traces(showlegend=False)

figure.show()

минимум f'' = 2.681638760023334
максимум f'' = 2.7173560908995227


### Вычисление $R_1(x)$ и определение минимума и максимума на отрезке $[i; i + 1]$

#### Вычисление $\omega_2(x)$ и определение минимума и максимума на отрезке $[i; i + 1]$

Для определения максимума и минимума $\omega_2(x)$ необходимо решить неравенство $\omega_2'(x) = 0$

In [8]:
omega2 = (x - x_points[i_2]) * (x - x_points[i_2 + 1])
omega2_1 = sympy.diff(omega2, x)
extremes_omega2 = sympy.solve(sympy.Eq(omega2_1, 0), x)

print("ω_2 =", omega2)
print("ω_2' =", omega2_1)
print("точки экстремума в ω_2:", extremes_omega2)

ω_2 = (x - 0.8)*(x - 0.75)
ω_2' = 2*x - 1.55
точки экстремума в ω_2: [0.775000000000000]


Для определения минимума и максимума на отрезке $[i; i + 1]$ необходимо рассмотреть значения функции $\omega_2(x)$ в точках экстремума, которые принадлежат отрезку, и на краях отрезка

In [9]:
x_omega2 = [x_points[i_2], x_points[i_2 + 1]] + [float(c) for c in extremes_omega2 if c.is_real and x_points[i_2] <= c <= x_points[i_2 + 1]]
y_omega2 = [float(omega2.subs(x, c)) for c in x_omega2]
omega2_min = min(y_omega2)
omega2_max = max(y_omega2)

print("минимум ω_2 =", omega2_min)
print("максимум ω_2 =", omega2_max)

figure = px.line()
for point in range(len(x_omega2)):
    figure.add_scatter(x=[x_omega2[point]], y=[y_omega2[point]], marker=dict(color="blue", size=5, symbol="circle"))
figure.update_traces(showlegend=False)

figure.show()

минимум ω_2 = -0.0006250000000000011
максимум ω_2 = 0.0


#### Вычисление $R_1(x)$ и определение минимума и максимума на отрезке $[i; i + 1]$

In [10]:
y_R_1 = [
    f2_min * omega2_min, f2_min * omega2_max,
    f2_max * omega2_min, f2_max * omega2_max
]
R_1_min = min(y_R_1) * 0.5
R_1_max = max(y_R_1) * 0.5

print("минимум R_1 =", R_1_min)
print("максимум R_1 =", R_1_max)

минимум R_1 = -0.0008491737784061023
максимум R_1 = 0.0


#### Проверка выполнения условий

In [11]:
R_1 = f - L_1_Lagrange
print(R_1_min, R_1.subs(x, x_star), R_1_max)
print(R_1_min < R_1.subs(x, x_star) < R_1_max)
print(abs(R_1.subs(x, x_star)) <= 1e-4)

-0.0008491737784061023 -0.000809546253547122 0.0
True
False


# Вычисление $L_2(x*)$ и $R_2(x*)$ в форме Лагранжа

## Определение $x_0, x_1$ и $x_{-1}$ для интерполяции по 3-м точкам

In [12]:
i_3 = 0
for i in range(len(x_points) - 1):
    if x_points[i] <= x_star <= x_points[i + 1]:
        if i == 0:
            i_3 = 1
            break

        if i == len(x_points) - 2:
            i_3 = len(x_points) - 2
            break

        if x_star - x_points[i] <= x_points[i + 1] - x_star:
            i_3 = i
        else:
            i_3 = i + 1
        break

    if i == len(x_points) - 2:
        raise ValueError("x* вне отрезка [a, b]")
    
print(x_points[i_3 - 1], x_points[i_3], x_points[i_3 + 1])
print(x_star)

figure = px.line(x=x_points, y=y_points, markers=True, color_discrete_sequence=["rgb(120, 120, 120)"])
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=[x_points[i_2 - 1]], y=[y_points[i_2 - 1]], name="x_-1", marker=dict(color="blue", size=5, symbol="circle"))
figure.add_scatter(x=[x_points[i_2]], y=[y_points[i_2]], name="x_0", marker=dict(color="blue", size=5, symbol="circle"))
figure.add_scatter(x=[x_points[i_2 + 1]], y=[y_points[i_2 + 1]], name="x_1", marker=dict(color="blue", size=5, symbol="circle"))

figure.add_scatter(x=[x_star], y=[float(f.subs(x, x_star))], name="x*", marker=dict(color="purple", size=6, symbol="circle"))

figure.show()

0.7 0.75 0.8
0.77


## Вычисление $L_2(x*)$

Интерполяционная формула Лагранжа для 3-х узлов имеет вид:

$\displaystyle L_2(x*) = f(x_{i - 1}) \frac{(x* - x_i) (x - x_{i + 1})}{(x_{i - 1} - x_i) (x_{i - 1} - x{i + 1})}+ f(x_i) \frac{(x* - x_{i - 1})(x* - x_{i + 1})}{(x_i - x_{i - 1}) (x_i - x_{i + 1})} + f(x_{i + 1}) \frac{(x* - x_{i - 1}) (x* - x_i)}{(x_{i + 1} - x_{i - 1}) (x_{i + 1} - x_i)}$

In [13]:
L_2_Lagrange = f.subs(x, x_points[i_3 - 1]) * \
    (x - x_points[i_3]) * (x - x_points[i_3 + 1]) / \
    (x_points[i_3 - 1] - x_points[i_3]) / (x_points[i_3 - 1] - x_points[i_3 + 1]) + \
    f.subs(x, x_points[i_3]) * \
    (x - x_points[i_3 - 1]) * (x - x_points[i_3 + 1]) / \
    (x_points[i_3] - x_points[i_3 - 1]) / (x_points[i_3] - x_points[i_3 + 1]) + \
    f.subs(x, x_points[i_3 + 1]) * \
    (x - x_points[i_3 - 1]) * (x - x_points[i_3]) / \
    (x_points[i_3 + 1] - x_points[i_3 - 1]) / (x_points[i_3 + 1] - x_points[i_3])
print("L_2 =" , L_2_Lagrange.subs(x, x_star))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_2_Lagrange.subs(x, c)) for c in x_points], name="L_2 Lagrange")

figure.show()

L_2 = -0.103230141402944


## Вычисление $R_2(x*)$

### Вычисление 3-й производной от $f(x)$ и определение минимума и максимума на отрезке $[i - 1; i + 1]$

Для определения максимума и минимума $f'''(x)$ необходимо решить неравенство $f^{IV}(x) = 0$

In [14]:
f4 = sympy.diff(f3, x)
extremes_f3 = sympy.solve(sympy.Eq(f4, 0), x)

print("f''' =", f3)
print("f'''' =", f4)
print("точки экстремума в f''':", extremes_f3)

f''' = cos(x)
f'''' = -sin(x)
точки экстремума в f''': [0, pi]


Для определения минимума и максимума на отрезке $[i - 1; i + 1]$ необходимо рассмотреть значения функции $f'''(x)$ в точках экстремума, которые принадлежат отрезку, и на краях отрезка

In [15]:
x_f3 = [x_points[i_3 - 1], x_points[i_3 + 1]] + [float(c) for c in extremes_f3 if c.is_real and x_points[i_3 - 1] <= c <= x_points[i_3 + 1]]
y_f3 = [float(f3.subs(x, c)) for c in x_f3]
f3_min = min(y_f3)
f3_max = max(y_f3)

print("минимум f''' =", f3_min)
print("максимум f''' =", f3_max)

figure = px.line()
for point in range(len(x_f3)):
    figure.add_scatter(x=[x_f3[point]], y=[y_f3[point]], marker=dict(color="blue", size=5, symbol="circle"))
figure.update_traces(showlegend=False)

figure.show()

минимум f''' = 0.6967067093471654
максимум f''' = 0.7648421872844885


### Вычисление $R_2(x)$ и определение минимума и максимума на отрезке $[i - 1; i + 1]$

#### Вычисление $\omega_3(x)$ и определение минимума и максимума на отрезке $[i - 1; i + 1]$

Для определения максимума и минимума $\omega_3(x)$ необходимо решить неравенство $\omega_3'(x) = 0$

In [16]:
omega3 = (x - x_points[i_3 - 1]) * (x - x_points[i_3]) * (x - x_points[i_3 + 1])
omega3_1 = sympy.diff(omega3, x)
extremes_omega3 = sympy.solve(sympy.Eq(omega3_1, 0), x)

print("ω_3 =", omega3)
print("ω_3' =", omega3_1)
print("точки экстремума в ω_3:", extremes_omega3)

ω_3 = (x - 0.8)*(x - 0.75)*(x - 0.7)
ω_3' = (x - 0.8)*(x - 0.75) + (x - 0.8)*(x - 0.7) + (x - 0.75)*(x - 0.7)
точки экстремума в ω_3: [0.721132486540519, 0.778867513459481]


Для определения минимума и максимума на отрезке $[i - 1; i + 1]$ необходимо рассмотреть значения функции $\omega_3(x)$ в точках экстремума, которые принадлежат отрезку, и на краях отрезка

In [17]:
x_omega3 = [x_points[i_3 - 1], x_points[i_3 + 1]] + [float(c) for c in extremes_omega3 if c.is_real and x_points[i_3 - 1] <= c <= x_points[i_3 + 1]]
y_omega3 = [float(omega3.subs(x, c)) for c in x_omega3]
omega3_min = min(y_omega3)
omega3_max = max(y_omega3)

print("минимум ω_3 =", omega3_min)
print("максимум ω_3 =", omega3_max)

figure = px.line()
for point in range(len(x_omega3)):
    figure.add_scatter(x=[x_omega3[point]], y=[y_omega3[point]], marker=dict(color="blue", size=5, symbol="circle"))
figure.update_traces(showlegend=False)

figure.show()

минимум ω_3 = -4.811252243246894e-05
максимум ω_3 = 4.8112522432468945e-05


#### Вычисление $R_2(x)$ и определение минимума и максимума на отрезке $[i - 1; i + 1]$

In [18]:
y_R_2 = [
    f3_min * omega3_min, f3_min * omega3_max,
    f3_max * omega3_min, f3_max * omega3_max
]
R_2_min = min(y_R_2) / 6
R_2_max = max(y_R_2) / 6

print("минимум R_2 =", R_2_min)
print("максимум R_2 =", R_2_max)

минимум R_2 = -6.13308114883726e-06
максимум R_2 = 6.133081148837261e-06


#### Проверка выполнения условий

In [19]:
R_2 = f - L_2_Lagrange
print(R_2_min, R_2.subs(x, x_star), R_2_max)
print(R_2_min < R_2.subs(x, x_star) < R_2_max)
print(abs(R_2.subs(x, x_star)) <= 1e-5)

-6.13308114883726e-06 -5.09722441258478e-6 6.133081148837261e-06
True
True


# Расчёт таблицы разделённых разностей

Разделённая разность n-го порядка рассчитывается по формуле:

$\displaystyle f(x_i, x_{i + 1}, ..., x_{i + n}) = \frac{f(x_{i + 1}, x_{i + 2}, ..., x_{i + n}) - f(x_i, x_{i + 1}, ..., x_{i + n - 1})}{x_{i + n} - x_i}$

Причём конечная разность 1-го порядка:

$\displaystyle f(x_i, x_{i + 1}) = \frac{f{x_{i + 1}} - f(x_i)}{x_{i + 1} - x_i}$

In [20]:
f_01 = (y_points[i_3] - y_points[i_3 - 1]) / (x_points[i_3] - x_points[i_3 - 1])
f_12 = (y_points[i_3 + 1] - y_points[i_3]) / (x_points[i_3 + 1] - x_points[i_3])
f_012 = (f_12 - f_01) / (x_points[i_3 + 1] - x_points[i_3 - 1])

divided_differences_table = pandas.DataFrame({
    "x": x_points[i_3 - 1: i_3 + 2],
    "0-й порядок": y_points[i_3 - 1: i_3 + 2],
    "1-й порядок": [None, f_01, f_12],
    "2-й порядок": [None, None, f_012]
})
divided_differences_table


,x,0-й порядок,1-й порядок,2-й порядок
0,0.70,-0.154218,NaN,NaN
1,0.75,-0.119139,0.701579,NaN
2,0.80,-0.077356,0.835653,1.340748


# Вычисление $L_1(x*)$ и $R_1(x*)$ в форме Ньютона

## Вычисление $L_1(x*)$

Интерполяционная формула Ньютона для 2-х узлов имеет вид:

$\displaystyle L_1(x*) = f(x_i) + f(x_i, x_{i + 1}) (x* - x_i)$

In [21]:
L_1_Newton = f.subs(x, x_points[i_2]) + \
    (y_points[i_2 + 1] - y_points[i_2]) / (x_points[i_2 + 1] - x_points[i_2]) * \
    (x - x_points[i_2])
print("L_1 =", L_1_Newton.subs(x, x_star))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_1_Newton.subs(x, c)) for c in x_points], name="L_1 Newton")

figure.show()

L_1 = -0.102425692373810


## Вычисление $R_1(x*)$ и проверка условий

In [22]:
R_1 = f - L_1_Newton
print(R_1_min, R_1.subs(x, x_star), R_1_max)
print(R_1_min < R_1.subs(x, x_star) < R_1_max)
print(abs(R_1.subs(x, x_star)) <= 1e-4)

-0.0008491737784061023 -0.000809546253547122 0.0
True
False


# Вычисление $L_2(x*)$ и $R_2(x*)$ в форме Ньютоне

## Вычисление $L_2(x*)$

Интерполяционная формула Лагранжа для 3-х узлов имеет вид:

$\displaystyle L_2(x*) = f(x_{i - 1}) + f(x_{i - 1}, x_i) (x* - x_{i - 1}) + f(x_{i - 1}, x_i, x_{i + 1}) (x* - x_{i - 1}) (x* - x_i)$

In [23]:
L_2_Newton = f.subs(x, x_points[i_3 - 1]) + \
    f_01 * (x - x_points[i_3 - 1]) + \
    f_012 * (x - x_points[i_3 - 1]) * (x - x_points[i_3])
print("L_2 =", L_2_Newton.subs(x, x_star))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_2_Newton.subs(x, c)) for c in x_points], name="L_2 Newton")

figure.show()

L_2 = -0.103230141402944


## Вычисление $R_2(x*)$

In [24]:
R_2 = f - L_2_Newton
print(R_2_min, R_2.subs(x, x_star), R_2_max)
print(R_2_min < R_2.subs(x, x_star) < R_2_max)
print(abs(R_2.subs(x, x_star)) <= 1e-5)

-6.13308114883726e-06 -5.09722441278384e-6 6.133081148837261e-06
True
True


# Сравнение полиномов Лагранжа и Ньютона

## Сравнение формул Лагранжа и Ньютона 1-го порядка

In [25]:
figure = px.line(x=x_points, y=[float(L_1_Lagrange.subs(x, c)) for c in x_points], markers=True)
figure.update_traces(name="L_1 Lagrange", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_1_Newton.subs(x, c)) for c in x_points], name="L_1 Newton")

figure.show()

## Сравнение формул Лагранжа и Ньютона 1-го порядка

In [26]:
figure = px.line(x=x_points, y=[float(L_2_Lagrange.subs(x, c)) for c in x_points], markers=True)
figure.update_traces(name="L_2 Lagrange", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_2_Newton.subs(x, c)) for c in x_points], name="L_2 Newton")

figure.show()